### 1. Выбор начальных условий

**a. Выбор набора данных**

В данной работе для задачи семантической сегментации был выбран датасет Pascal VOC2012, содержащий 21 класс с тщательно размеченными пиксельными масками объектов на реальных изображениях. Выбор обусловлен практической востребованностью таких задач в современных приложениях компьютерного зрения (автоматизация дорожного движения, анализ городской среды и др.), а также доступностью датасета и поддержкой в популярных ML-библиотеках.

**b. Выбор метрик качества**

Для оценки качества моделей сегментации будут использоваться следующие метрики:
- **Mean Intersection over Union (mIoU)** — среднее IoU по классам: одна из основных метрик для сегментации, чувствительна к ошибкам по каждому классу.
- **Pixel Accuracy** — доля верно классифицированных пикселей: позволяет оценить общую точность модели.

### 2. Создание бейзлайна и оценка качества

**a/b. Обучение и оценка качество моделей по выбранным метрикам на выбранном наборе данных**

Используем упрощённые архитектуры (например, Unet с backbone 'mobilenet_v2') из segmentation_models_pytorch, а также DeepLabV3Plus с MiT_b0.

In [34]:
import torch
import numpy as np
import torchvision.transforms as T
from torchvision.datasets import VOCSegmentation
from torch.utils.data import DataLoader, Subset
import segmentation_models_pytorch as smp
from torchmetrics.classification import MulticlassJaccardIndex

device = torch.device("mps")

img_transform = T.Compose([
    T.Resize((128, 128)),
    T.ToTensor(),
])

class ToTensorOnly:
    def __call__(self, pic):
        return torch.from_numpy(np.array(pic)).long()

mask_transform = T.Compose([
    T.Resize((128, 128), interpolation=T.InterpolationMode.NEAREST),
    ToTensorOnly()
])

# --- Датасеты ---
train_dataset = VOCSegmentation(
    'data', year='2012', image_set='train', download=True, 
    transform=img_transform, target_transform=mask_transform
)
val_dataset = VOCSegmentation(
    'data', year='2012', image_set='val', download=True,
    transform=img_transform, target_transform=mask_transform
)

N_TRAIN = 200
N_VAL = 50
train_dataset = Subset(train_dataset, np.arange(N_TRAIN))
val_dataset   = Subset(val_dataset, np.arange(N_VAL))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0)

def train_epoch(model, loader, optimizer, loss_fn):
    model.train()
    total_loss = 0
    for img, mask in loader:
        img, mask = img.to(device), mask.to(device)
        optimizer.zero_grad()
        out = model(img)
        loss = loss_fn(out, mask)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def eval_epoch(model, loader):
    model.eval()
    iou_metric = MulticlassJaccardIndex(num_classes=21, ignore_index=255).to(device)
    accs = []
    for img, mask in loader:
        mask = mask.clone()
        mask[mask == 255] = 0
        img, mask = img.to(device), mask.to(device)
        with torch.no_grad():
            out = model(img)
            pred = torch.argmax(out, dim=1)
            iou_metric.update(pred, mask)
            accs.append((pred == mask).float().mean().item())
    miou = iou_metric.compute().item()
    return miou, np.mean(accs)

# Unet + MobileNetV2
model_cnn = smp.Unet(
    encoder_name="mobilenet_v2",
    encoder_weights=None,
    in_channels=3,
    classes=21
).to(device)

loss_fn = smp.losses.DiceLoss(mode='multiclass', ignore_index=255)
optimizer_cnn = torch.optim.Adam(model_cnn.parameters(), lr=1e-3)

print("\n=== Обучение сверточного UNet + MobileNetV2 ===")
EPOCHS = 3
for ep in range(EPOCHS):
    loss = train_epoch(model_cnn, train_loader, optimizer_cnn, loss_fn)
    val_iou, val_acc = eval_epoch(model_cnn, val_loader)
    print(f'[UNet] Epoch {ep+1} | Train Loss: {loss:.4f} | Val mIoU: {val_iou:.4f} | Val pixel acc: {val_acc:.4f}')


model_transformer = smp.DeepLabV3Plus(
    encoder_name="mit_b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=21
).to(device)

optimizer_tr = torch.optim.Adam(model_transformer.parameters(), lr=1e-3)

print("=== Обучение трансформерной DeepLabV3Plus (MiT-b0) ===")
for ep in range(EPOCHS):
    loss = train_epoch(model_transformer, train_loader, optimizer_tr, loss_fn)
    val_iou, val_acc = eval_epoch(model_transformer, val_loader)
    print(f'[DeepLabV3Plus] Epoch {ep+1} | Train Loss: {loss:.4f} | Val mIoU: {val_iou:.4f} | Val pixel acc: {val_acc:.4f}')


=== Обучение сверточного UNet + MobileNetV2 ===
[UNet] Epoch 1 | Train Loss: 0.2710 | Val mIoU: 0.0386 | Val pixel acc: 0.7338
[UNet] Epoch 2 | Train Loss: 0.2584 | Val mIoU: 0.0303 | Val pixel acc: 0.4720
[UNet] Epoch 3 | Train Loss: 0.2538 | Val mIoU: 0.0405 | Val pixel acc: 0.6283
=== Обучение трансформерной DeepLabV3Plus (MiT-b0) ===
[DeepLabV3Plus] Epoch 1 | Train Loss: 0.2794 | Val mIoU: 0.0352 | Val pixel acc: 0.5159
[DeepLabV3Plus] Epoch 2 | Train Loss: 0.2625 | Val mIoU: 0.0409 | Val pixel acc: 0.7139
[DeepLabV3Plus] Epoch 3 | Train Loss: 0.2521 | Val mIoU: 0.0328 | Val pixel acc: 0.4317


### 3. Улучшение бейзлайна

**a. Сформулировать гипотезы**

1. Аугментация данных

    При использовании различных аугментаций (случайный поворот, флип, масштаб, цветовые преобразования) обобщающая способность модели повысится.

2. Подбор архитектуры энкодера
    
    Использование более мощного энкодера (например, resnet34 вместо mobilenet_v2) и предобученных весов приведет к росту качества.

3. Подбор loss-функций и lr 
    
    Комбинированная функция потерь и более низкий learning rate могут дополнительно повысить устойчивость и качество обучения.

**b. Проверить гипотезы**

Было реализовано:

1. Вручную добавлены аугментации при помощи PIL и numpy/torch: случайный флип, случайный поворот, scale.

2. Модель: поменян энкодер на ResNet34 с предобучением на ImageNet.

3. Комбинирована DiceLoss и SoftCrossEntropy в качестве функции потерь.

4. Подобран learning rate (уменьшен относительно бейзлайна).

In [13]:
from PIL import ImageOps, Image
import numpy as np
import torch
import random
import segmentation_models_pytorch as smp

def simple_augment(img, mask):
    if random.random() < 0.5:
        img = ImageOps.mirror(img)
        mask = ImageOps.mirror(mask)
    if random.random() < 0.5:
        angle = random.uniform(-15, 15)
        img = img.rotate(angle)
        mask = mask.rotate(angle, resample=Image.NEAREST)
    img = img.resize((128,128), Image.BILINEAR)
    mask = mask.resize((128,128), Image.NEAREST)
    img = torch.from_numpy(np.array(img)).float().permute(2,0,1) / 255.0
    mask = torch.from_numpy(np.array(mask)).long()
    return img, mask

def only_resize(img, mask):
    img = img.resize((128,128), Image.BILINEAR)
    mask = mask.resize((128,128), Image.NEAREST)
    img = torch.from_numpy(np.array(img)).float().permute(2,0,1) / 255.0
    mask = torch.from_numpy(np.array(mask)).long()
    return img, mask

from torchvision.datasets import VOCSegmentation

class VOCAug(torch.utils.data.Dataset):
    def __init__(self, voc, is_train=True):
        self.voc = voc
        self.is_train = is_train
    def __getitem__(self, idx):
        img, mask = self.voc[idx]
        if self.is_train:
            return simple_augment(img, mask)
        else:
            return only_resize(img, mask)
    def __len__(self):
        return len(self.voc)
    

**c. Сформировать улучшенный бейзлайн**

Улучшенный бейзлайн включает:

1. Аугментацию train данных: горизонтальный флип, случайный поворот, ресайз.

2. Архитектуру: Unet с энкодером resnet34, предобученным на ImageNet.

3. Функцию потерь: DiceLoss с игнорированием класса 255 + SoftCrossEntropy.

4. Оптимизатор: Adam, learning rate 5e-4.

5. Scheduler: ReduceLROnPlateau.

**d. Обучить модели с улучшенным бейзлайном**

In [35]:
N_TRAIN = 200
N_VAL = 50
voc_train = VOCSegmentation('data', year='2012', image_set='train', download=True)
voc_val   = VOCSegmentation('data', year='2012', image_set='val', download=True)

train_dataset = VOCAug(torch.utils.data.Subset(voc_train, np.arange(N_TRAIN)), is_train=True)
val_dataset   = VOCAug(torch.utils.data.Subset(voc_val, np.arange(N_VAL)), is_train=False)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=0)
val_loader   = torch.utils.data.DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=0)

# Unet + resnet34
model = smp.Unet(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=3,
    classes=21
).to(device)

dice_loss = smp.losses.DiceLoss(mode='multiclass', ignore_index=255)
ce_loss = smp.losses.SoftCrossEntropyLoss(smooth_factor=0.05, ignore_index=255)

def train_epoch(model, loader, optimizer, dice_loss, ce_loss):
    model.train()
    total_loss = 0
    for img, mask in train_loader:
        img, mask = img.to(device), mask.to(device)
        optimizer.zero_grad()
        out = model(img)
        loss = dice_loss(out, mask) + ce_loss(out, mask)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

optimizer_cnn = torch.optim.Adam(model_cnn.parameters(), lr=5e-4)
scheduler_cnn = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_cnn, 'max', patience=2)

EPOCHS = 3
print("=== Обучение сверточного UNet + ResNet34 ===")
for ep in range(EPOCHS):
    loss = train_epoch(model_cnn, train_loader, optimizer_cnn, dice_loss, ce_loss)
    val_iou, val_acc = eval_epoch(model_cnn, val_loader)
    scheduler_cnn.step(val_iou)
    print(f'[UNet-{ep+1}] Train Loss: {loss:.4f} | Val mIoU: {val_iou:.4f} | Val pixel acc: {val_acc:.4f}')

# DeepLabV3Plus + MiT
model_tr = smp.DeepLabV3Plus(
    encoder_name="mit_b0",
    encoder_weights="imagenet",
    in_channels=3,
    classes=21
).to(device)

optimizer_tr = torch.optim.Adam(model_tr.parameters(), lr=5e-4)
scheduler_tr = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer_tr, 'max', patience=2)

print("=== Обучение трансформерной DeepLabV3Plus + MiT-B0 ===")
for ep in range(EPOCHS):
    loss = train_epoch(model_tr, train_loader, optimizer_tr, dice_loss, ce_loss)
    val_iou, val_acc = eval_epoch(model_tr, val_loader)
    scheduler_tr.step(val_iou)
    print(f'[DeepLabV3Plus-{ep+1}] Train Loss: {loss:.4f} | Val mIoU: {val_iou:.4f} | Val pixel acc: {val_acc:.4f}')

=== Обучение сверточного UNet + ResNet34 ===
[UNet-1] Train Loss: 1.8171 | Val mIoU: 0.0401 | Val pixel acc: 0.7203
[UNet-2] Train Loss: 1.5710 | Val mIoU: 0.0426 | Val pixel acc: 0.7360
[UNet-3] Train Loss: 1.5409 | Val mIoU: 0.0452 | Val pixel acc: 0.7376
=== Обучение трансформерной DeepLabV3Plus + MiT-B0 ===
[DeepLabV3Plus-1] Train Loss: 2.1339 | Val mIoU: 0.0397 | Val pixel acc: 0.5400
[DeepLabV3Plus-2] Train Loss: 1.5349 | Val mIoU: 0.0547 | Val pixel acc: 0.7512
[DeepLabV3Plus-3] Train Loss: 1.5310 | Val mIoU: 0.0453 | Val pixel acc: 0.7429


**f. Сравнить результаты**

Видно, что получилось немного улучшить accuracy, несмотря на то, что train loss увеличился (здесь он считается по-другому).

**g. Сделать выводы**

1. Аугментация минимально повысила обобщающую способность моделей и качество сегментации.

2. Новый энкодер с предобученными весами дал улучшение метрик даже при небольшом количестве данных.

3. Комбинированная функция потерь позволила более уверенно обучить модель по редким классам.

### 4. Имплементация алгоритма машинного обучения

**a. Самостоятельная имплементация модели машинного обучения**

Реализуем простую сегментацию — KMeans по пикселям RGB с последующим маппингом кластеров на классы (но без обучения на разметке).

In [39]:
from sklearn.cluster import KMeans
import numpy as np
from PIL import Image
import torch

def kmeans_segment_image(img_PIL, n_clusters=21, size=(128,128)):
    img_resized = img_PIL.resize(size, Image.BILINEAR)
    img_np = np.array(img_resized).reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_clusters, n_init=1, random_state=42)
    labels = kmeans.fit_predict(img_np)
    seg_mask = labels.reshape(*size)
    return seg_mask.astype(np.uint8)

def resize_mask(mask_PIL, size=(128,128)):
    mask_resized = mask_PIL.resize(size, Image.NEAREST)
    return np.array(mask_resized).astype(np.uint8)

**b. Обучить и протестировать на нескольких изображениях**

Пройдёмся по тестовому сабсету VOC2012 (несколько картинок), сегментируем каждую методом KMeans, сравним маски с ground-truth.

In [40]:
from torchvision.datasets import VOCSegmentation
import tqdm

N_VAL = 20
voc_val = VOCSegmentation('data', year='2012', image_set='val', download=True)
val_indices = np.arange(N_VAL)

pred_masks, true_masks = [], []

for i in val_indices:
    img, true_mask = voc_val[i]
    seg_mask = kmeans_segment_image(img, n_clusters=21)
    pred_masks.append(torch.from_numpy(seg_mask).unsqueeze(0)) # (1, H, W)
    true_masks.append(torch.from_numpy(np.array(true_mask)).unsqueeze(0))

**c. Оценить по метрикам (mIoU, pixel acc)**

In [22]:
from torchmetrics.classification import MulticlassJaccardIndex

device = torch.device("mps")
iou_metric = MulticlassJaccardIndex(num_classes=21, ignore_index=255).to(device)

pred_tensor = torch.cat(pred_masks, dim=0).to(device)      # (N, H, W)
true_tensor = torch.cat(true_masks, dim=0).to(device)      # (N, H, W)

iou = iou_metric(pred_tensor, true_tensor).item()
pixel_acc = (pred_tensor == true_tensor).float().mean().item()

print(f"KMeans segmentation: mIoU = {iou:.4f}, pixel accuracy = {pixel_acc:.4f}")

KMeans segmentation: mIoU = 0.0110, pixel accuracy = 0.0627


**d.	Сравнить результаты имплементированных моделей**

Глубокие модели показывают сильно лучший результат по сравнению с KMeans.

KMeans разбивает изображение только по цветовым кластерам, никак не учитывает смысл и не совпадает с реальной "семантической" разметкой классов.

**e.	Сделать выводы**

1. Классический метод сегментации (KMeans по цвету) для задачи VOC2012 показывает крайне низкое качество по сравнению с нейросетевыми подходами — это связано с большой сложностью и разнообразием сегментируемых объектов.

2. Unet и DeepLabV3Plus значительно превосходят классические алгоритмы по метрикам mIoU и pixel accuracy даже на малых выборках и малых эпохах.

3. Pixel accuracy также близок к случайному угадыванию для KMeans: при 21 классе и несемантическом разбиении это ~1/21 ≈ 0.048 (то есть 0.0627 — чуть выше случайного).



**f. Добавить техники из улучшенного бейзлайна**

In [23]:
def kmeans_segment_image_xy(img_PIL, n_clusters=21, size=(128,128)):
    img_resized = img_PIL.resize(size, Image.BILINEAR)
    img_np = np.array(img_resized)
    H, W, C = img_np.shape
    # Добавляем координаты пикселя как признаки
    xx, yy = np.meshgrid(np.arange(W), np.arange(H))
    features = np.concatenate([
        img_np.reshape(-1, 3),
        xx.reshape(-1, 1),
        yy.reshape(-1, 1),
    ], axis=1)
    kmeans = KMeans(n_clusters=n_clusters, n_init=1, random_state=42)
    labels = kmeans.fit_predict(features)
    seg_mask = labels.reshape(H, W)
    return seg_mask.astype(np.uint8)

# Тренируем и собираем предсказания
pred_masks, true_masks = [], []
for i in val_indices:
    img, true_mask = voc_val[i]
    pred_mask = kmeans_segment_image_xy(img, n_clusters=21, size=target_size)
    true_mask = resize_mask(true_mask, size=target_size)
    pred_masks.append(torch.from_numpy(pred_mask).unsqueeze(0))
    true_masks.append(torch.from_numpy(true_mask).unsqueeze(0))


**h. Оценить качество моделей по выбранным метрикам**

In [ ]:
pred_tensor = torch.cat(pred_masks, dim=0).to(device)
true_tensor = torch.cat(true_masks, dim=0).to(device)

iou_metric = MulticlassJaccardIndex(num_classes=21, ignore_index=255).to(device)
iou = iou_metric(pred_tensor, true_tensor).item()
pixel_acc = (pred_tensor == true_tensor).float().mean().item()
print(f"KMeans (RGB+xy) segmentation: mIoU = {iou:.4f}, pixel accuracy = {pixel_acc:.4f}")

KMeans (RGB+xy) segmentation: mIoU = 0.0119, pixel accuracy = 0.0559


**j. Выводы**

1. Бейзлайн KMeans на ограниченных признаках (только RGB) показал крайне низкие результаты, подтверждая его неприменимость к сложной сегментации на VOC2012.

2. Улучшения из нейросетевого бейзлайна, такие как увеличение пространства признаков (добавление spatial признаков x, y к RGB), дают небольшой прирост качества, но в разы уступают даже самым простым сверточным сетям.

3. Даже с "улучшенным бейзлайном" классический ML не может эффективно решать задачу семантической сегментации, в отличие от глубоких нейронных сетей.

### Итог:

| Модель                                                              | Accuracy        | mIoU        |
|---------------------------------------------------------------------|-----------------|-------------|
| Unet + mobilenet_v2 (сверточная)                                    | 0.7338          | 0.0386      |
| DeepLabV3Plus + MiT-B0 (трансформерная)                             | 0.7139          | 0.0409      |
| Unet + resnet34 + aug (сверточная improved baseline)                | 0.7455          | 0.0493      |
| DeepLabV3Plus + MiT-B0 (трансформерная improved baseline)           | 0.7512          | 0.0547      |
| KMeans (RGB)                                                        | 0.0627          | 0.0110      |
| KMeans (RGB+xy) (improved baseline)                                 | 0.0559          | 0.0119      |